In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re

### Get a user's playlists

In [2]:
def get_plst_ID(user_id, token):
    tidy = lambda s: s[17:]
    base_url = "https://api.spotify.com/v1/users/"
    query = f'{base_url}{user_id}/playlists'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['uri'])for x in json_response['items']]

### Get a Playlist's Items

In [3]:
def get_ID(user_id, token, playlist_id, market = 'US'):
    tidy = lambda s: s[14:]
    base_url = "https://api.spotify.com/v1/playlists/"
    query = f'{base_url}{playlist_id}/tracks?market={market}'

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    return [tidy(x['track']['uri'])for x in json_response['items']], [json_response['items'][i]['track']['duration_ms'] for i in range(len(json_response['items']))]

### Get information about a song

In [4]:
def get_song(user_id, token, song_id, market = 'US'):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    response = requests.get(query, 
                   headers={"Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    album_id = json_response['album']['id']
    artist_id = json_response['artists'][0]['id']
    
    ## get genre of the album
    base_url = "https://api.spotify.com/v1/albums/"
    query = f'{base_url}{album_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    genre = json_response['genres']
    
    
    df_response['artist_id'] = artist_id if len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if len(album_id) >0 else None
    df_response['genre'] = genre if len(genre) >0 else None
    
    return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')

### Get Recommendations from Spotify API (Nov 7 updated)

In [5]:
def check(par, par_range):
    ''' Check if values are in the feasible range. '''
    return(np.clip(par, par_range[0], par_range[1]))

In [6]:
def check_whole(kwargs):
    ''' Run the above check function to all optional parameters. '''
    pars = ['valence', 'speechiness', 'acousticness', 'liveness', 'danceability', 'hahaha']
    ranges = {'valence': [0,1], 'speechiness': [0,1], 'acousticness': [0,1], 'liveness': [0,1], 'danceability': [0,1]}
    if len(kwargs) == 0:
        return(kwargs)
    else:
        for item in pars:
            if np.array([re.match(f"(.*?)({item})", x) is not None for x in kwargs.keys()]).any(): 
                item_min = 'min_' + item
                item_max = 'max_' + item
                item_target = 'target_' + item
                par_range = ranges.get(item)
                kwargs[item_target] = check(kwargs.get(item_target), par_range)
                kwargs[item_min] = check(kwargs.get(item_min), par_range)
                kwargs[item_max] = check(kwargs.get(item_max), par_range)
        return(kwargs)

In [7]:
def recommend(song, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
    seed_tracks = song.index[0]
    
    time_mt = require_percents.get('time')
    energy_percent = require_percents.get('energy')
    instrumentalness_percent = require_percents.get('instrumentalness')
    tempo_percent = require_percents.get('tempo')
    
    min_duration_ms = max(target_duration_ms - time_mt * 1000, 0)
    max_duration_ms = target_duration_ms + time_mt * 1000
    
    ## required percents
    target_energy = song['energy'][0]
    min_energy = check(target_energy * (1 - energy_percent), [0,1])
    max_energy = check(target_energy * (1 + energy_percent), [0,1])
    target_instrumentalness = song['instrumentalness'][0]
    min_instrumentalness = check(target_instrumentalness * (1 - instrumentalness_percent), [0,1])
    #max_instrumentalness = check(target_instrumentalness * (1 + instrumentalness_percent), [0,1])
    target_tempo = song['tempo'][0]
    min_tempo = check(target_tempo * (1 - tempo_percent), [0, np.inf])
    max_tempo = check(target_tempo * (1 + tempo_percent), [0, np.inf])

    ## default parameters
    target_key = song['key'][0]
    target_danceability=song['danceability'][0]
    target_mode=song['mode'][0]
         
    ## optional percents
    for item in optional_percents: 
        item_min = 'min_' + item
        item_max = 'max_' + item
        item_target = 'target_' + item
        target = song[item][0]
        kwargs[item_target] = target
        if item not in ['mode', 'key', 'popularity', 'time_signature']:
            kwargs[item_min] = target * (1 - optional_percents.get(item))
            kwargs[item_max] = target * (1 + optional_percents.get(item))
    kwargs.pop('optional_percents', None)   ## remove `optional_percents` from kwargs
    kwargs = check_whole(kwargs)
    
    endpoint_url = "https://api.spotify.com/v1/recommendations?"
    query = f'{endpoint_url}limit={limit}&seed_tracks={seed_tracks}'
    query += f'&target_duration_ms={target_duration_ms}&min_duration_ms={min_duration_ms}&max_duration_ms={max_duration_ms}'
    query += f'&target_energy={target_energy}&min_energy={min_energy}&max_energy={max_energy}'
    query += f'&target_instrumentalness={target_instrumentalness}&min_instrumentalness={min_instrumentalness}'
    query += f'&target_tempo={target_tempo}&min_tempo={min_tempo}&max_tempo={max_tempo}'
    query += f'&target_key={target_key}'
    query += f'&target_danceability={target_danceability}'
    query += f'&target_mode={target_mode}'
    
    if song['genre'][0] is not None:
        seed_genres = song['genre'][0]
        query += f'&seed_genres={seed_genres}'
    if song['artist_id'][0] is not None:
        seed_artist = song['artist_id'][0]
        query += f'&seed_artist={seed_artist}'

    if len(kwargs.keys()) > 0:
        query += '&'
        lst = [str(x[0])+'='+str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all limits from kwargs
    uris = [] 
    #print(query)
    
    response = requests.get(query, 
               headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    json_response = response.json()
    #print('Recommended Songs:')
    uris = []
    for i,j in enumerate(json_response['tracks']):
        uris.append(j['uri'])
        #print(f"{i+1}) \"{j['name']}\" by {j['artists'][0]['name']}")
    return uris

### Filter recommended songs

In [8]:
def filter_energy(r, pre_energy, market, trend = None):
    ''' filter recommended songs according to the energy trend '''
    
    if trend is None or pre_energy == 0:
        return r[0]
    
    energys_diff = [get_song(user_id, token, r[i][14:], market)['energy'][0] - pre_energy for i in range(len(r))]
    if trend == '+':
        if all(np.array(energys_diff) <= 0):
            return None
        idx = np.argmax( np.array(energys_diff) > 0)
    else:
        if all(np.array(energys_diff) >= 0):
            return None
        idx = np.argmax( np.array(energys_diff) < 0)
    return(r[idx])

### Create a playlist

In [9]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

### Generate a similar playlist

In [10]:
def get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, public=False, **kwargs):
    print(playlist_id)
    
    market = kwargs.get("market", 'US')
    kwargs["market"] = market
    
    percent_args = kwargs.get("percent_args", {})
    requires = ['time', 'energy', 'instrumentalness', 'tempo']
    require_percents = {}
    optional_percents = {}
    for item in requires:  ## for required percents, extract them and store in `require_percents`
        require_percents[item] = percent_args.get(item, 0.1)
    for item in list(set(percent_args) - set(requires)): ## for optional percents, store it in `optional_percents`
        optional_percents[item] = percent_args.get(item)
    kwargs.pop('percent_args', None)        
    print(kwargs, require_percents, optional_percents)
    
    def _get_one_recommendation(idx, limit, target_duration_ms, require_percents, optional_percents, **kwargs):
        df_response = get_song(user_id, token, idx, market)
        r = recommend(df_response, limit, target_duration_ms, require_percents, optional_percents, **kwargs)

        print(f"For the {j+1}-th song, no recommendations, enlarged the range about duration.")
        new_time_percent = require_percents.get('time')
        new_energy_percent = require_percents.get('energy')
        new_require_percents = require_percents.copy()
        i = 0
        while len(r) == 0 and i < 10:
            i += 1
            new_time_percent += 10
            new_energy_percent *= 2
            new_require_percents['time'] = new_time_percent
            #new_require_percents['energy'] = new_energy_percent
            r = recommend(df_response, limit, target_duration_ms, new_require_percents, optional_percents, **kwargs)
        if i == 10:
            ## right now I keep the original one, we can improve it later
            r = ['spotify:track:' + df_response.index[0]]
            ## another way is to remove all optional limitations
            # r = recommend(df_response, limit, target_duration_ms, require_percents, {}, **kwargs)
        return r
        
    uris = []
    duration = 0
    ids, durations = get_ID(user_id, token, playlist_id, market)
    diff = 0
    pre_energy = 0
    ene = 0
    trend = None
    for j in range(len(ids)):
        energy = get_song(user_id, token, ids[j], market)['energy'][0] 
        trend = '+' if energy > pre_energy else '-'
        _limit = limit
        _r = None
        i = 0
        while _r is None and i < 10:
            ## if no songs satisfy requirements, we get more recommendations from API and then filter
            r = _get_one_recommendation(ids[j], _limit, durations[j]-diff, require_percents, optional_percents, **kwargs)    
            #print(ene)
            #print(trend)
            if len(r) < _limit:
                break
            _r = filter_energy(r, ene, market, trend) ## add limitations to filter the recommendations and keep the energy trend
            _limit = _limit + 10
            i = i+1
        if i == 10 or len(r) < _limit:
            _r = r[0] ## if still no songs satisfy requirements, use the most similar song from Spotify Recommendation API

        print('recommendation: ', _r[14:])
        uris.append(_r)
        dur = get_song(user_id, token, _r[14:], market)['duration_ms'][0] ## duration of the recommended song
        duration += dur
        diff = duration - sum(durations[:j+1])
        ene = get_song(user_id, token, _r[14:], market)['energy'][0] ## energy of the recommended song
        pre_energy = energy  ## energy of the j-th song in the playlist
    
    create_plst(user_id, token, [x for x in uris if x is not None], name, description, public)
    return([x[14:] for x in uris if x is not None])

## Test

Note: token should have scope at least "playlist-modify-private", you can find get one at the botton GET TOKEN from [here](https://developer.spotify.com/console/get-recommendations/?limit=10&market=ES&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical%2Ccountry&seed_tracks=0c6xIDDpzE81m2q797ordA&min_acousticness=&max_acousticness=&target_acousticness=&min_danceability=&max_danceability=&target_danceability=&min_duration_ms=&max_duration_ms=&target_duration_ms=&min_energy=&max_energy=&target_energy=&min_instrumentalness=&max_instrumentalness=&target_instrumentalness=&min_key=&max_key=&target_key=&min_liveness=&max_liveness=&target_liveness=&min_loudness=&max_loudness=&target_loudness=&min_mode=&max_mode=&target_mode=&min_popularity=&max_popularity=&target_popularity=&min_speechiness=&max_speechiness=&target_speechiness=&min_tempo=&max_tempo=&target_tempo=&min_time_signature=&max_time_signature=&target_time_signature=&min_valence=&max_valence=&target_valence=).

User id can be obtained from [your Spotify profile](https://www.spotify.com/us/account/overview/?utm_source=spotify&utm_medium=menu&utm_campaign=your_account).

In [11]:
# settings
token = "BQCeeccYOwM0dG_P2tAwRPyNoHBHm9e3U3on2wzW4sygzzs85irk8t91IonjHgyXhRkuNFaBESnlHJ7S4ajcVcsu84YnbMsEVL7OW4bZRV1usV_pvIAxjlqUe9xOrimQh6TQhVs2v5B80SwhfQHoA8eyPeOE8FGCZlYtISbwk6O87UZMEYNIK_-Mya1DRXlzIqnnofLIpLx70w"
user_id = "pbwppse1hilahmk43ls424ao4"

name = 'new method no energy 6/21'   ## name for the new playlist
description = 'new method' ## description for the new playlist
#f'recommendations based on {re.match("(.*?)Recommandation", name).group(1)}' 
limit = 10  ## for each song in current playlist, 10 recommendations are generated

plst_id = get_plst_ID(user_id, token)  ## get id for each playlist from a user
#playlist_id = plst_id[7]    ## set the 2nd playlist to be the target playlist

In [12]:
playlist_id = '4N2UHn9HpFc3n93s1gduIM'

In [13]:
percent_args = {'time': 15, 'energy': 0.1} #'liveness': 0.05

In [14]:
# run it
recommend_pl = get_recommendation_for_a_playlist(user_id, token, playlist_id, name, description, limit, market='US', percent_args = percent_args)

4N2UHn9HpFc3n93s1gduIM
{'market': 'US'} {'time': 15, 'energy': 0.1, 'instrumentalness': 0.1, 'tempo': 0.1} {}
For the 1-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  7bzks4LGpQUuPKBzJ6iQ7y
For the 2-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  7nBR4Tt431p1MTgv3lVsmX
For the 3-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  6Fkdtw5xR9OgK8G62l0xjs
For the 4-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  4Uw7NtaXY0xJrbR9qiaN4H
For the 5-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  5860DNiiXsxmcRrZquqNH4
For the 6-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  6GYD9MwygCfPurS4Dd7uvT
For the 7-th song, no recommendations, enlarged the range about duration and energy.
recommendation:  7caJcFZTtLzy0ZSol1AXKH
For the 8-th so

#### Check your new playlist in your Spotify!

### Measure of Similarity

In [15]:
pars = ['energy', 'instrumentalness', 'tempo', 'key', 'liveness', 'loudness', 'acousticness', 'danceability', 'mode', 
        'speechiness', 'time_signature', 'valence']

In [16]:
def weighted_metric(weight, par_name, original_playlist, recommend_playlist):
    original = 0
    recommend = 0
    ori_weight = 0
    rec_weight = 0
    for j in range(len(original_playlist)):
        ori_song = get_song(user_id, token, original_playlist[j], market).iloc[0]
        ori_weight += ori_song[weight]
        original += ori_song[par_name] * ori_song[weight]
        #print('weight', song[weight], '\n par', song[par_name])
        rec_song = get_song(user_id, token, recommend_playlist[j], market).iloc[0]
        rec_weight += rec_song[weight]
        recommend += rec_song[par_name] * rec_song[weight]

    return((recommend/rec_weight - original/ori_weight), original/ori_weight, recommend/rec_weight)

In [21]:
market = 'US'
ori_ids, ori_durations = get_ID(user_id, token, playlist_id, market)

metrics = pd.DataFrame(index = pars, columns = ['difference', 'original', 'recommendations'])
for i in range(len(pars)):
    print(pars[i])
    metrics.loc[pars[i], :] = weighted_metric('duration_ms', pars[i], ori_ids, recommend_pl)

energy
instrumentalness
tempo
key
liveness
loudness
acousticness
danceability
mode
speechiness
time_signature
valence


In [22]:
metrics

,difference,original,recommendations
energy,0.01381,0.685684,0.699494
instrumentalness,0.19987,0.472374,0.672244
tempo,-0.750569,120.125,119.374
key,-1.82023,6.74098,4.92075
liveness,0.034828,0.117799,0.152627
loudness,0.506385,-7.79164,-7.28525
acousticness,-0.0148573,0.18191,0.167052
danceability,-0.00535384,0.722525,0.717171
mode,-0.220894,0.69677,0.475875
speechiness,0.00767977,0.0464536,0.0541334


The difference between the original playlist and the recommendations seems trivial.